## *PhotoDissociation Region Toolbox Notebooks*
-------------------------------------------------------------

# Example 1: Working With Measurements

This Example shows how to bring your spectral line and far-infrared (FIR) data into the PDR Toolbox.  Your data can be single pixel values or spatial maps (FITS files). 

In [ ]:
# First import the relevant modules.
from pdrtpy.measurement import Measurement
from pdrtpy.modelset import ModelSet
import pdrtpy.pdrutils as utils
from astropy.nddata import StdDevUncertainty
import astropy.units as u
import numpy as np

In [ ]:
# check that your notebooks align with your pdrtpy version
utils.check_nb()

## Measurements
To use PDR Toolbox, you need to create [`Measurements`](https://pdrtpy.readthedocs.io/en/latest/pdrtpy.measurement.html) from your observations. A `Measurement` consists of a value and an error.  These can be single-valued or an array of values.  In the typical case of an image, the `Measurement` is a representation of a FITS file with two Header Data Units (HDUs), the first HDU is the spatial map of intensity and the 2nd HDU is the spatial map of the errors.  It is based on [astropy's CCDData](https://docs.astropy.org/en/stable/api/astropy.nddata.CCDData.html). Typical sub-millimeter maps we get from telescopes don't have the error plane, but PDRT makes it easy for you to create one if you know the magnitude of the error. Your FITS images can be in intensity units (equivalent to erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$) or can be in K km/s.  PDRT will do appropriate conversion as necessary when it uses your images (original `Measurement` remains unchanged).

### Measurement Identifiers 
When you create a `Measurement` you have to say what it is a measurement of, i.e., what spectral line it is.  This is done using the string identifier.  The identifier (ID) should be one of the lines supported by the PDR Toolbox [`ModelSets`](https://pdrtpy.readthedocs.io/en/latest/pdrtpy.modelset.html).   The default `ModelSet` in the PDR Toolbox is the Wolfire/Kaufman 2006 constant density models.  See the `ModelSet` example notebook for more about models and what's available.

In [ ]:
ModelSet("wk2006",z=1).supported_lines

## 1. Creating a Measurement based on a single value

Suppose you have single-beam observations of [OI] 145 $\mu$m, [CI] 609 $\mu$m, CO(J=2-1), and [CII] 158 $\mu$m lines. You create Measurements for these using the constructor giving it the value, error, line identifier string, and units.  The value and the error must be in the same units. You can add optional beam size (bmaj,bmin,bpa), but note PDRT requires all Measurements have the same beam size before it can perform calculations.  (If you don't provide beam parameters for any of your Measurements, PDRT will assume they are all the same).

In [ ]:
myunit = "erg s-1 cm-2 sr-1" # this is my default for value and error
m1 = Measurement(data=30.,uncertainty = StdDevUncertainty(5.0),identifier="OI_145",unit=myunit)
m2 = Measurement(data=10.,uncertainty = StdDevUncertainty(2.0),identifier="CI_609",unit=myunit)
m3 = Measurement(data=10.,uncertainty = StdDevUncertainty(1.5),identifier="CO_21",unit=myunit)
m4 = Measurement(data=100.,uncertainty = StdDevUncertainty(10.),identifier="CII_158",unit=myunit,bmaj=60*u.arcsec,bmin=60*u.arcsec,bpa=0*u.degree)
print(m1)
# You can convert any Measurement to equivalent units 
w = utils.to("W m^-2 sr^-1",m1)
print(w)
# Print the beam major axis value
print("bmaj (deg): %.3e"%m4.header["BMAJ"])

## 2. Creating an array of Measurements from a Table
Instantiation of many Measurements can be done quickly by listing them 
in an external table and reading the table with the `Measurement.from_table` method.
Tables can be in any [astropy recognized format](https://docs.astropy.org/en/stable/io/unified.html#built-in-table-readers-writers).  Required columns are *identifier, data, uncertainty* with optional columns *bmaj, bmin, bpa* following the inputs to the `Measurement` constructor.    Based the unit in the table header, the *uncertainty* will be interpreted as absolute or as a percentage of the *data*.  Included with these notebooks are two example table in IPAC format: `measurement_example1.tab` has *uncertainty* as percent and  `measurement_example2.tab` has beam parameters.

`Measurement.from_table` can return either a single `Measurement` where the data array contains all rows of the table, or Python list of `Measurements` with one Measurement per table row.  This is controlled with the keyword `array`. 

In [ ]:
# Read in a table where all rows are put into one measurement.
a = Measurement.from_table("../data/measurement_example1.tab",array=False)
print(type(a))
print(f"{a:3.5f}")

# Read in a table where each row is put into its own Measurement and the list is returned
a2 = Measurement.from_table("../data/measurement_example2.tab",array=True)
print(type(a2))
for m in a2:
    print(m.id,m)

## 3. Creating Measurements from a FITS images

Typical sub-millimeter maps we get from telescopes don't have the error plane, but PDRT makes it easy for you to create one if you know the magnitude of the error or if you have a separate error map. Your FITS images can be in intensity units (equivalent to erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$) or can be in K km s$^{-1}$.  PDRT will do appropriate conversion as necessary when it reads in your images.   Here is an example using the N22 data from [Jameson et al. 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...853..111J/abstract). These data consist of separate flux and error maps for [OI] 145 $\mu$m and [CII] 158 $\mu$m,  and a flux map of the total far-infrared emission (FIR) which has a 10% error.  The static method `Measurement.make_measurement` will put the flux and error together in a single image with 2 HDUs and write that FITS file to disk.  You then read the image back in via the `read` method.

*Note:* This example locates in the test data distributed with these `pdrtpy` using a shortcut `utils.get_testdata`.  In more typical operation, you would simply provide the full-qualified pathnames to your FITS data files. 

In [ ]:
# Get the input filenames of the FITS files in the testdata directory
# These are maps from Jameson et al 2018.
print("Test FITS files are in: %s"%utils.testdata_dir())
cii_flux = utils.get_testdata("n22_cii_flux.fits")  # [C II] flux
cii_err = utils.get_testdata("n22_cii_error.fits")  # [C II] error
oi_flux = utils.get_testdata("n22_oi_flux.fits")    # [O I] flux 
oi_err = utils.get_testdata("n22_oi_error.fits")    # [O I] error
FIR_flux = utils.get_testdata("n22_FIR.fits")       # FIR flux

# Output file names
cii_combined = "n22_cii_flux_error.fits"
oi_combined = "n22_oi_flux_error.fits"
FIR_combined = "n22_FIR_flux_error.fits"

# create the Measurements and write out the FITS files.
# Set overwrite=True to allow multiple runs of this notebook.
Measurement.make_measurement(cii_flux, cii_err, cii_combined,overwrite=True)
Measurement.make_measurement(oi_flux, oi_err, oi_combined,overwrite=True)
# Assign a 10% error in FIR flux
Measurement.make_measurement(FIR_flux, error='10%', outfile=FIR_combined,overwrite=True)

# Read in the FITS files to Measurements
cii_meas = Measurement.read(cii_combined, identifier="CII_158")
FIR_meas = Measurement.read(FIR_combined, identifier="FIR")
oi_meas = Measurement.read(oi_combined, identifier="OI_63")

print(oi_meas.wcs)
print("[OI] max %.2E min %.2E  %s"%(np.nanmax(oi_meas.data), np.nanmin(oi_meas.data),oi_meas.unit))

## The more you have in the FITS header the less you have to specify

Here we read in the CO(J=1-0) integrated intensity map that has the units, error, and beam parameters specified by the FITS keywords BUNIT, RMS, BMAJ, BMIN, BPA respectively.  These are old Bell Labs 7-m data of the Draco High Latitude Cloud (MBM 41-44).


In [ ]:
outfile="draco_CO_measurement.fits"
infile=utils.get_testdata("draco.ico.fits")
Measurement.make_measurement(infile,error='rms',outfile=outfile,overwrite=True)
ico = Measurement.read(outfile,identifier="CO_10")
print(ico.unit)
ico.header